#### In this notebook we will import the original DC rejetive data as a csv file and create the weather station id mapping information. ToDO: hook up with Azure blob container to automatically receive the csv file from the Azure blob container

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyspark
from functools import reduce
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import desc,asc, monotonically_increasing_id, split, size, length

from IPython.display import display, HTML

In [17]:
%run /Users/p0z00cx/jupter/product_weather_prediction/lib/production_utility.ipynb

In [2]:
data = spark.read.csv('/Users/p0z00cx/hactron_product/rejection_data.csv', header="true", inferSchema="true")
display(data.toPandas())

,ITEM_NBR,ITEM_DESCRIPTION,FINELINE_NBR,FINELINE_DESC,DEPT_CATEGORY_DESC,VENDOR_NAME,VENDOR_NBR,INSPECTION_DATE,INSPECTED_CASES,INSPECTION_RESULT,...,PROCESS_DAYS,PACK_OR_HARVEST_DATE,MANUAL_CREATE_IND,IMPORT_IND,FRT_PYMNT_TYP_DESC,ORDERED_CASES,RECEIVED_CASES,Shipping City,SHIP STATE,SHIPPING ZIP CODE
0,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,None,NaN,None,...,4,None,N,N,COLLECT,117,117,None,California,NaN
1,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,9/10/19,117.0,aminus,...,2,None,N,N,PREPAID,117,117,None,California,NaN
2,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,9/9/19,74.0,rejected,...,2,None,Y,N,PREPAID,74,0,None,California,NaN
3,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,9/11/19,117.0,aminus,...,5,None,N,N,COLLECT,117,117,None,California,NaN
4,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,None,NaN,None,...,2,None,N,N,PREPAID,234,234,None,California,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159081,580941213,ORG BLKBERRY 6OZ,3511,BLACKBERRY ORGANIC 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,185256,None,NaN,None,...,4,None,N,N,PREPAID,108,0,McCallen,TX,78504.0
159082,580941213,ORG BLKBERRY 6OZ,3511,BLACKBERRY ORGANIC 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,185256,None,NaN,None,...,4,None,N,N,PREPAID,108,0,McCallen,TX,78504.0
159083,580941213,ORG BLKBERRY 6OZ,3511,BLACKBERRY ORGANIC 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,185256,None,NaN,None,...,4,None,N,N,PREPAID,108,0,McCallen,TX,78504.0
159084,580941213,ORG BLKBERRY 6OZ,3511,BLACKBERRY ORGANIC 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,185256,None,NaN,None,...,4,None,N,N,PREPAID,108,0,McCallen,TX,78504.0


Update the names of columns

In [3]:
col_list=[
  'item_num', 
  'item_desc', 
  'fineline_num',
  'fineline_desc',
  'dept_desc',
  'vendor_name',
  'vendor_num',
  'inspection_date',
  'inspected_cases',
  'inspected_result',
  'rejection_reason',
  'DC_num',
  'order_date',
  'vendor_9d_num',
  'rejected_cases',
  'minus_quality_cases',
  'country',
  'state',
  'region',
  'po_num',
  'po_line_num',
  'dept_num',
  'avg_core_temp',
  'min_ship_temp',
  'max_ship_temp',
  'dc_name',
  'city_name',
  'state_code',
  'vm_yr_wk',
  'mabd_date',
  'not_ship_date',
  'process_days',
  'pack_day',
  'manual_ind',
  'import_ind',
  'pymnt_typ',
  'ordered_cases',
  'received_cases',
  'ship_city',
  'ship_state',
  'ship_zip'
 ]

In [4]:
oldColumns = data.schema.names
newColumns = col_list
data = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)),data)
display(data.toPandas())

,item_num,item_desc,fineline_num,fineline_desc,dept_desc,vendor_name,vendor_num,inspection_date,inspected_cases,inspected_result,...,process_days,pack_day,manual_ind,import_ind,pymnt_typ,ordered_cases,received_cases,ship_city,ship_state,ship_zip
0,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,None,NaN,None,...,4,None,N,N,COLLECT,117,117,None,California,NaN
1,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,9/10/19,117.0,aminus,...,2,None,N,N,PREPAID,117,117,None,California,NaN
2,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,9/9/19,74.0,rejected,...,2,None,Y,N,PREPAID,74,0,None,California,NaN
3,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,9/11/19,117.0,aminus,...,5,None,N,N,COLLECT,117,117,None,California,NaN
4,9402156,BLACKBERRY 12OZ,3501,BLACKBERRIES 12OZ,BLACKBERRIES,DRISCOLLS INC,865951,None,NaN,None,...,2,None,N,N,PREPAID,234,234,None,California,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159081,580941213,ORG BLKBERRY 6OZ,3511,BLACKBERRY ORGANIC 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,185256,None,NaN,None,...,4,None,N,N,PREPAID,108,0,McCallen,TX,78504.0
159082,580941213,ORG BLKBERRY 6OZ,3511,BLACKBERRY ORGANIC 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,185256,None,NaN,None,...,4,None,N,N,PREPAID,108,0,McCallen,TX,78504.0
159083,580941213,ORG BLKBERRY 6OZ,3511,BLACKBERRY ORGANIC 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,185256,None,NaN,None,...,4,None,N,N,PREPAID,108,0,McCallen,TX,78504.0
159084,580941213,ORG BLKBERRY 6OZ,3511,BLACKBERRY ORGANIC 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,185256,None,NaN,None,...,4,None,N,N,PREPAID,108,0,McCallen,TX,78504.0


In [5]:
data_region  = data.select("state", "region").distinct()
display(data_region .toPandas())

,state,region
0,Federal District,Chao
1,Colima,SMR/CMR Region
2,Coquimbo,Coquimbo
3,California,Central
4,Michoacan,Jalisco
...,...,...
241,Veracruz,Veracruz
242,California,Oxnard
243,Colima,Peru
244,Veracruz,Martinez de la Torre


In [6]:
# map the state and region with weather station id
state_station_id = spark.read.csv('/Users/p0z00cx/hactron_product/region_state_stationid.csv', header="true", inferSchema="true",)
display(state_station_id.toPandas())

,Colima,SMR/CMR Region,MXM00076658
0,California,Central,BR000970000
1,Michoacan,Jalisco,MXN00014101
2,Tala,Jalisco,AJ000037575
3,Michoacan,mexico,MXN00014101
4,Lima,Lima,ASN00010303
...,...,...,...
204,Michoacan,Mexico,MXN00014101
205,Queretaro,North Central,MXM00076625
206,California,Oxnard,BR000970000
207,Colima,Peru,MXM00076658


In [7]:
col_list=['state', 'region', 'station_id']
oldColumns = state_station_id.schema.names
newColumns = col_list
state_station_id = reduce(lambda state_station_id, idx: state_station_id.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), state_station_id)
display(state_station_id.toPandas())

,state,region,station_id
0,California,Central,BR000970000
1,Michoacan,Jalisco,MXN00014101
2,Tala,Jalisco,AJ000037575
3,Michoacan,mexico,MXN00014101
4,Lima,Lima,ASN00010303
...,...,...,...
204,Michoacan,Mexico,MXN00014101
205,Queretaro,North Central,MXM00076625
206,California,Oxnard,BR000970000
207,Colima,Peru,MXM00076658


In [8]:
from pyspark.sql.functions import concat, lit
state_station_id = state_station_id.withColumn("state_region_key", concat(state_station_id.state, state_station_id.region))
display(state_station_id.toPandas())

,state,region,station_id,state_region_key
0,California,Central,BR000970000,CaliforniaCentral
1,Michoacan,Jalisco,MXN00014101,MichoacanJalisco
2,Tala,Jalisco,AJ000037575,TalaJalisco
3,Michoacan,mexico,MXN00014101,Michoacanmexico
4,Lima,Lima,ASN00010303,LimaLima
...,...,...,...,...
204,Michoacan,Mexico,MXN00014101,MichoacanMexico
205,Queretaro,North Central,MXM00076625,QueretaroNorth Central
206,California,Oxnard,BR000970000,CaliforniaOxnard
207,Colima,Peru,MXM00076658,ColimaPeru


In [29]:
need_data = data.select('item_desc', 'dept_desc', 'vendor_name','inspected_result', 'order_date','country','state','region','dc_name','mabd_date', 'ordered_cases').filter(data['inspected_result'] !='null')
need_data = need_data.withColumn("state_region_key", concat(need_data.state, need_data.region))
need_data = need_data.drop("state").drop("region")
display(need_data.toPandas())

,item_desc,dept_desc,vendor_name,inspected_result,order_date,country,dc_name,mabd_date,ordered_cases,state_region_key
0,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/4/19,United States of America (USA),"COLTON, CA FRESH",9/10/19,117,CaliforniaWatsonville
1,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,rejected,9/3/19,United States of America (USA),"COLTON, CA FRESH",9/9/19,74,CaliforniaWatsonville
2,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/3/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/11/19,117,CaliforniaWatsonville
3,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/2/19,United States of America (USA),"OCHELATA, OK GDC",9/8/19,234,CaliforniaWatsonville
4,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/6/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/14/19,351,CaliforniaWatsonville
...,...,...,...,...,...,...,...,...,...,...
26165,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/5/20,Mexico,"LEWISTON, ME GDC",4/13/20,108,JaliscoMexico
26166,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"BEDFORD, PA GDC",4/24/20,108,JaliscoMexico
26167,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"JOHNSTOWN, NY GDC",4/23/20,108,JaliscoMexico
26168,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/12/20,Mexico,"JOHNSTOWN, NY GDC",4/19/20,216,JaliscoMexico


In [30]:
need_data_with_station_id = need_data.join(state_station_id, 'state_region_key', 'left')
need_data_with_station_id = need_data_with_station_id.drop('state_region_key')
display(need_data_with_station_id.toPandas())

,item_desc,dept_desc,vendor_name,inspected_result,order_date,country,dc_name,mabd_date,ordered_cases,state,region,station_id
0,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/4/19,United States of America (USA),"COLTON, CA FRESH",9/10/19,117,California,Watsonville,BR000970000
1,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,rejected,9/3/19,United States of America (USA),"COLTON, CA FRESH",9/9/19,74,California,Watsonville,BR000970000
2,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/3/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/11/19,117,California,Watsonville,BR000970000
3,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/2/19,United States of America (USA),"OCHELATA, OK GDC",9/8/19,234,California,Watsonville,BR000970000
4,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/6/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/14/19,351,California,Watsonville,BR000970000
...,...,...,...,...,...,...,...,...,...,...,...,...
26165,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/5/20,Mexico,"LEWISTON, ME GDC",4/13/20,108,Jalisco,Mexico,MXN00014179
26166,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"BEDFORD, PA GDC",4/24/20,108,Jalisco,Mexico,MXN00014179
26167,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"JOHNSTOWN, NY GDC",4/23/20,108,Jalisco,Mexico,MXN00014179
26168,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/12/20,Mexico,"JOHNSTOWN, NY GDC",4/19/20,216,Jalisco,Mexico,MXN00014179


In [31]:
need_data_with_station_id = need_data_with_station_id.filter(need_data_with_station_id["mabd_date"].isNotNull())
display(need_data_with_station_id.toPandas())

,item_desc,dept_desc,vendor_name,inspected_result,order_date,country,dc_name,mabd_date,ordered_cases,state,region,station_id
0,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/4/19,United States of America (USA),"COLTON, CA FRESH",9/10/19,117,California,Watsonville,BR000970000
1,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,rejected,9/3/19,United States of America (USA),"COLTON, CA FRESH",9/9/19,74,California,Watsonville,BR000970000
2,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/3/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/11/19,117,California,Watsonville,BR000970000
3,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/2/19,United States of America (USA),"OCHELATA, OK GDC",9/8/19,234,California,Watsonville,BR000970000
4,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/6/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/14/19,351,California,Watsonville,BR000970000
...,...,...,...,...,...,...,...,...,...,...,...,...
26165,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/5/20,Mexico,"LEWISTON, ME GDC",4/13/20,108,Jalisco,Mexico,MXN00014179
26166,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"BEDFORD, PA GDC",4/24/20,108,Jalisco,Mexico,MXN00014179
26167,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"JOHNSTOWN, NY GDC",4/23/20,108,Jalisco,Mexico,MXN00014179
26168,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/12/20,Mexico,"JOHNSTOWN, NY GDC",4/19/20,216,Jalisco,Mexico,MXN00014179


In [32]:
need_data_with_station_id = need_data_with_station_id.withColumn("start_day", get_start_day_udf("mabd_date"))
display(need_data_with_station_id.toPandas())

,item_desc,dept_desc,vendor_name,inspected_result,order_date,country,dc_name,mabd_date,ordered_cases,state,region,station_id,start_day
0,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/4/19,United States of America (USA),"COLTON, CA FRESH",9/10/19,117,California,Watsonville,BR000970000,20190821
1,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,rejected,9/3/19,United States of America (USA),"COLTON, CA FRESH",9/9/19,74,California,Watsonville,BR000970000,20190820
2,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/3/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/11/19,117,California,Watsonville,BR000970000,20190822
3,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/2/19,United States of America (USA),"OCHELATA, OK GDC",9/8/19,234,California,Watsonville,BR000970000,20190819
4,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/6/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/14/19,351,California,Watsonville,BR000970000,20190825
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26165,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/5/20,Mexico,"LEWISTON, ME GDC",4/13/20,108,Jalisco,Mexico,MXN00014179,20200324
26166,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"BEDFORD, PA GDC",4/24/20,108,Jalisco,Mexico,MXN00014179,20200404
26167,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"JOHNSTOWN, NY GDC",4/23/20,108,Jalisco,Mexico,MXN00014179,20200403
26168,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/12/20,Mexico,"JOHNSTOWN, NY GDC",4/19/20,216,Jalisco,Mexico,MXN00014179,20200330


In [34]:
need_data_with_station_id = need_data_with_station_id.withColumn("end_day", get_end_day_udf("mabd_date"))
display(need_data_with_station_id.toPandas())

,item_desc,dept_desc,vendor_name,inspected_result,order_date,country,dc_name,mabd_date,ordered_cases,state,region,station_id,start_day,end_day
0,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/4/19,United States of America (USA),"COLTON, CA FRESH",9/10/19,117,California,Watsonville,BR000970000,20190821,20190910
1,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,rejected,9/3/19,United States of America (USA),"COLTON, CA FRESH",9/9/19,74,California,Watsonville,BR000970000,20190820,20190909
2,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/3/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/11/19,117,California,Watsonville,BR000970000,20190822,20190911
3,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/2/19,United States of America (USA),"OCHELATA, OK GDC",9/8/19,234,California,Watsonville,BR000970000,20190819,20190908
4,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/6/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/14/19,351,California,Watsonville,BR000970000,20190825,20190914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26165,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/5/20,Mexico,"LEWISTON, ME GDC",4/13/20,108,Jalisco,Mexico,MXN00014179,20200324,20200413
26166,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"BEDFORD, PA GDC",4/24/20,108,Jalisco,Mexico,MXN00014179,20200404,20200424
26167,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"JOHNSTOWN, NY GDC",4/23/20,108,Jalisco,Mexico,MXN00014179,20200403,20200423
26168,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/12/20,Mexico,"JOHNSTOWN, NY GDC",4/19/20,216,Jalisco,Mexico,MXN00014179,20200330,20200419


In [35]:
need_data_with_station_id.toPandas().to_csv('/Users/p0z00cx/hactron_product/rejection_data_with_station_id.csv', header="true", index=None)